# Data Analysis of Sabine Hossenfelder YT' videos
## This is Part 1 - steps I've taken to go from a xls file of raw data  (copied and pasted from SH YT'channels) to a clean dataframe
## Part 2 is EDA 
## Part 3 is my attempt at modeling the data

Excel file contains data scraped (copy&paste) from Sabine Hossenfelder YT channel.
https://www.youtube.com/@SabineHossenfelder
I need to perform some data cleaning and reformatting, since my file is just two 1-columns sheets, containing

*   the duration of video (rows no. 1, 4, 7 )
*   the title (rows no. 2, 5, 8...) 
*   string consisting in number of views (number + K or number + M) and
*   time ago (number + day | days | week | weeks | month | months | year | years ago) (rows no. 3, 6, 9...)


The goal is to create a Data Frame with


1.   index ------------------------------------------ -- int
2.   title --------------------------------------------- -- string
3.   duration_min ------------------------------- -- int
4.   views -------------------------------------------- - int
5.   time ago-in days --------------------------  -- int
6.   date #need to find out how -------- -- date






In [5]:
import pandas as pd
import numpy as np

In [6]:
raw_videos = pd.read_excel('Sabine (1).xlsx')


In [7]:
raw_videos.head()

,Unnamed: 0
0,8:15NOW PLAYING
1,"We Need to Start Climate Engineering Soon, New..."
2,24K views4 hours ago
3,7:06NOW PLAYINGWatch laterAdd to queue
4,Record Quantum Computation at D-Wave: Millions...


In [8]:
video_list = list(raw_videos['Unnamed: 0'])

In [9]:
from datetime import datetime, timedelta

In [10]:
def get_duration(content):
  '''
  content - string
  returns: duration in min
  ignores seconds converst number of hours and minutes into minutes
  '''
  duration = 0
  content = content.replace('NOW PLAYING', '')
  d_list= content.split(':') # splits string into hours, minutes and seconds
  if len(d_list)<2:
    return 0
  if len(d_list)<3:
    return int(d_list[0])

  duration = int(d_list[0])*60 + int(d_list[1])
  return duration






In [11]:
def get_views(content):
  '''
    content: string containing the number of views as numberK or numberM
    return number of views in int
  '''
  frags = content.split(' views')
  if 'K' in frags[0]:
    views = float(frags[0].replace('K', ''))
    views = views *1000
  if 'M' in frags[0]:
    views =float(frags[0].replace('M', ''))
    views = views *1_000_000

  ago = frags[1].split()
  ago_str = ' '.join(ago)
  return views, ago_str


In [12]:

def decluter(raw_list):
  '''
  raw_list: list of data string
  returns: df of clean data
  '''
  durations = []
  views = []
  agos = []
  titles = []
  vid_idx = 0
  for index, content in enumerate(raw_list):
    if index%3 == 0:
      duration = get_duration(content)
      durations.append(duration)
    if index%3 == 1:
      titles.append(content)
    if index%3 == 2:
      no_views, ago = get_views(content)
      views.append(no_views)
      agos.append(ago)
  d = {'title': titles, 'duration': durations, 'views': views, 'ago': agos}
  df = pd.DataFrame(data=d)
  return df


In [13]:
clean_vids = decluter(video_list)

In [14]:
clean_vids.head()

,title,duration,views,ago
0,"We Need to Start Climate Engineering Soon, New...",8,24000.0,4 hours ago
1,Record Quantum Computation at D-Wave: Millions...,7,215000.0,1 day ago
2,New Idea Solves Three Physics Mysteries at Onc...,7,348000.0,2 days ago
3,"Meta Lawsuit Successful, Major Spam Domain Pro...",6,73000.0,3 days ago
4,"Dark Matter Could Form Stars, New Theory Says",6,122000.0,4 days ago


In [15]:
clean_vids.dtypes

title        object
duration      int64
views       float64
ago          object
dtype: object

In [16]:
clean_vids = clean_vids.astype({'views': int})

In [17]:
clean_vids.dtypes

title       object
duration     int64
views        int64
ago         object
dtype: object

In [18]:
import datetime

# Returns the current local date
today = datetime.date(2023, 11, 13)
print("Today is: {}".format(today))

Today is: 2023-11-13


In [19]:
def compute_timedelta(df):
  '''
  df - dataframe
  col - col with str with information how long ago a video has been uploaded or streamed
  returns: timedelta/ or date will see XD
  '''
  if 'hour' in df['ago']:
    delta_t = 0
  if 'day' in df['ago']:
    delta_t = int(df['ago'].split()[0])
  if 'week' in df['ago']:
    delta_t = int(df['ago'].split()[0]) * 7
  if 'month' in df['ago']:
    delta_t = int(df['ago'].split()[0]) * 30
  if 'year' in df['ago']:
    delta_t = int(df['ago'].split()[0]) * 365

  return delta_t

In [20]:
#create 'timedelta'

clean_vids['timedelta'] = clean_vids.apply(compute_timedelta, axis=1)


In [21]:
clean_vids['timedelta']

0         0
1         1
2         2
3         3
4         4
       ... 
431    3285
432    3285
433    3285
434    3650
435    4380
Name: timedelta, Length: 436, dtype: int64

In [22]:
def distribute_dates(df, col):
  '''
  df - dataframe
  col - column with grouped dates
  returns distributed dates from youtube's grouped descriptions
  for example if today is 14. Nov and 10 videos have been released 1 month ago it distributes the dates between 14. Oct and 14. Sept
  '''
  timedelta_group = df[col]
  timedelta_dis = []
  check = [0,0]

  for idx, num in enumerate(timedelta_group):
    if check[1]!=num:
      check[0] = 0
      check[1] = num

    num_vids = list(timedelta_group).count(num)


    if num >= 365:
      rate = 365/num_vids
      timedelta_dis.append(timedelta_group[idx] + (rate * check[0]))
      check[0] = check[0] + 1
      continue
    if num > 28:
      rate = 30/ num_vids
      timedelta_dis.append(timedelta_group[idx] + (rate * check[0]))
      check[0] = check[0] + 1
      continue
    if num > 7:
      rate = 7/num_vids
      timedelta_dis.append(timedelta_group[idx] + (rate * check[0]))
      check[0] = check[0] + 1
      continue
    if num <= 7:
      timedelta_dis.append(timedelta_group[idx])
      check[0] = check[0] + 1

  return timedelta_dis



In [23]:
timedelta_vid_dis = distribute_dates(clean_vids, 'timedelta')

In [24]:
def add_release_date(df, deltas):
  '''
  adds a column with distributed yt release dates
  '''
  df['date'] = today
  df['date'] = pd.to_datetime(df['date'])
  df['timedelta_dis'] = deltas
  df['delta'] = pd.to_timedelta(df['timedelta_dis'].apply(np.ceil), unit= 'days' )
  df['release_date'] = df['date'] - df['delta']

  return df

In [25]:
ts_clean_vids = add_release_date(clean_vids, timedelta_vid_dis)


In [26]:
ts_clean_vids.head()

,title,duration,views,ago,timedelta,date,timedelta_dis,delta,release_date
0,"We Need to Start Climate Engineering Soon, New...",8,24000,4 hours ago,0,2023-11-13,0.0,0 days,2023-11-13
1,Record Quantum Computation at D-Wave: Millions...,7,215000,1 day ago,1,2023-11-13,1.0,1 days,2023-11-12
2,New Idea Solves Three Physics Mysteries at Onc...,7,348000,2 days ago,2,2023-11-13,2.0,2 days,2023-11-11
3,"Meta Lawsuit Successful, Major Spam Domain Pro...",6,73000,3 days ago,3,2023-11-13,3.0,3 days,2023-11-10
4,"Dark Matter Could Form Stars, New Theory Says",6,122000,4 days ago,4,2023-11-13,4.0,4 days,2023-11-09


In [27]:
ts_clean_vids.drop(columns=['ago', 'timedelta', 'timedelta_dis', 'delta', 'date'], inplace=True)

In [28]:
ts_clean_vids.head()

,title,duration,views,release_date
0,"We Need to Start Climate Engineering Soon, New...",8,24000,2023-11-13
1,Record Quantum Computation at D-Wave: Millions...,7,215000,2023-11-12
2,New Idea Solves Three Physics Mysteries at Onc...,7,348000,2023-11-11
3,"Meta Lawsuit Successful, Major Spam Domain Pro...",6,73000,2023-11-10
4,"Dark Matter Could Form Stars, New Theory Says",6,122000,2023-11-09


In [29]:
ts_clean_vids.to_csv('SH_video_data.csv')
